# Download the file

In [22]:
!wget "http://eforexcel.com/wp/wp-content/uploads/2020/09/5m-Sales-Records.zip"

--2020-09-13 10:29:03--  http://eforexcel.com/wp/wp-content/uploads/2020/09/5m-Sales-Records.zip
Resolving eforexcel.com (eforexcel.com)... 162.214.80.24
Connecting to eforexcel.com (eforexcel.com)|162.214.80.24|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 201708941 (192M) [application/zip]
Saving to: ‘5m-Sales-Records.zip.2’

5m-Sales-Records.zi 100%[===================>] 192.36M  17.6MB/s    in 12s     

2020-09-13 10:29:16 (15.7 MB/s) - ‘5m-Sales-Records.zip.2’ saved [201708941/201708941]



In [23]:
import zipfile
import os
with zipfile.ZipFile("/content/5m-Sales-Records.zip", 'r') as zip_ref:
    zip_ref.extractall()

In [24]:
print ("Size of file is {} MB".format(os.path.getsize("/content/5m Sales Records.csv") >> 20))

Size of file is 595 MB


# Check GPU

In [25]:
!nvidia-smi

Sun Sep 13 10:29:21 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W /  70W |   4669MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Install RAPIDS

In [26]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
PLEASE READ
********************************************************************************************************
Changes:
1. IMPORTANT CHANGES: RAPIDS on Colab will be pegged to 0.14 Stable until further notice.
2. Default stable version is now 0.14.  Nightly will redirect to 0.14.
3. You can now declare your RAPIDSAI version as a CLI option and skip the user prompts (ex: '0.14' or '0.15', between 0.13 to 0.14, without the quotes): 
        "!bash rapidsai-csp-utils/colab/rapids-colab.sh <version/label>"
        Examples: '!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.14', or '!bash rapidsai-csp-utils/colab/rapids-colab.sh stable', or '!bash rapidsai-csp-utils/colab/rapids-colab.sh s'
                  '!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.15, or '!bash rapidsai-csp-utils/colab/rapids-colab.sh nightly', or '!bash rapidsai-csp-utils/colab/rapids-colab.sh n'
Enjoy using RAPIDS!  

In [27]:
import time
import numpy as np
import pandas as pd
import cudf

In [28]:
pd.set_option('display.float_format', lambda x: '%.1f' % x)

# Read Time Comparision

In [29]:
s = time.time()
df_pandas = pd.read_csv("/content/5m Sales Records.csv")
e = time.time()
pd_time = e - s
print("Pandas Loading Time = {}".format(pd_time))

Pandas Loading Time = 8.285161018371582


In [30]:
df_pandas.head()

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Australia and Oceania,Palau,Office Supplies,Online,H,3/6/2016,517073523,3/26/2016,2401,651.2,525.0,1563555.2,1260429.0,303126.2
1,Europe,Poland,Beverages,Online,L,4/18/2010,380507028,5/26/2010,9340,47.5,31.8,443183.0,296918.6,146264.4
2,North America,Canada,Cereal,Online,M,1/8/2015,504055583,1/31/2015,103,205.7,117.1,21187.1,12062.3,9124.8
3,Europe,Belarus,Snacks,Online,C,1/19/2014,954955518,2/27/2014,1414,152.6,97.4,215748.1,137780.2,77968.0
4,Middle East and North Africa,Oman,Cereal,Offline,H,4/26/2019,970755660,6/2/2019,7027,205.7,117.1,1445453.9,822932.0,622521.9


In [31]:
s = time.time()
df_cudf = cudf.read_csv("/content/5m Sales Records.csv")
e = time.time()
cuDF_time = e - s 
print("cuDF Loading Time = {}".format(cuDF_time))

cuDF Loading Time = 0.5019645690917969


In [32]:
pd_time > cuDF_time

True

In [33]:
pd_time / cuDF_time

16.505469765250368

# Concat Time Comparision

In [34]:
s = time.time()
df_pandas_concat = pd.concat([df_pandas for _ in range(5)])
e = time.time()
pd_time = e - s
print("Pandas Concat Time = {}".format(pd_time))

Pandas Concat Time = 4.841130256652832


In [35]:
s = time.time()
df_cudf_concat = cudf.concat([df_cudf for _ in range(5)])
e = time.time()
cuDF_time = e - s
print("cuDF Concat Time = {}".format(cuDF_time))

cuDF Concat Time = 0.09721565246582031


In [36]:
pd_time > cuDF_time

True

In [37]:
pd_time / cuDF_time

49.79784771135396

# GroupBy Time Comparision

In [38]:
s = time.time()
print(df_pandas.groupby("Region")["Total Profit"].sum())
e = time.time()
pd_time = e - s
print("Pandas GroupBy Calculation Time = {}".format(pd_time))

Region
Asia                                286686809544.5
Australia and Oceania               159289739668.2
Central America and the Caribbean   212358212842.6
Europe                              508972722261.1
Middle East and North Africa        244077925345.9
North America                        42491973495.3
Sub-Saharan Africa                  509522251166.1
Name: Total Profit, dtype: float64
Pandas GroupBy Calculation Time = 0.3099544048309326


In [39]:
s = time.time()
print(df_cudf.groupby("Region")["Total Profit"].sum())
e = time.time()
cuDF_time  = e - s
print("Pandas GroupBy Calculation Time = {}".format(cuDF_time))

Region
Asia                                286686809544.5
Australia and Oceania               159289739668.2
Central America and the Caribbean   212358212842.6
Europe                              508972722261.1
Middle East and North Africa        244077925345.9
North America                        42491973495.3
Sub-Saharan Africa                  509522251166.1
Name: Total Profit, dtype: float64
Pandas GroupBy Calculation Time = 0.06271958351135254


In [40]:
pd_time > cuDF_time

True

In [41]:
pd_time / cuDF_time

4.941907893486401